# Creating Pipelines For Model Training

### Step 1 => Importing Libraries And Dataset

In [31]:
import numpy as np
import pandas as pd

In [94]:
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.compose import ColumnTransformer 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

In [33]:
df = pd.read_csv("titanic.csv")

In [34]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [35]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

### Step 2 => Train/Test/Split

In [38]:
X_train,X_test,Y_train,Y_test=train_test_split(df.drop(columns=['Survived']), df['Survived'], test_size=0.2, random_state=32)

In [39]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
177,1,male,54.0,1,0,55.4417,C
245,1,male,41.0,1,0,51.8625,S
302,2,male,43.0,0,1,21.0000,S
355,1,male,50.0,0,0,26.0000,S
375,1,female,45.0,0,0,262.3750,C


In [40]:
Y_train.head()

177    0
245    0
302    0
355    0
375    1
Name: Survived, dtype: int64

### Step 3 => Column Transformation

In [41]:
# Imputation Transformer
trf1 = ColumnTransformer([
    ('impute_Age',SimpleImputer(),[2]),
    ('impute_Embarked',SimpleImputer(strategy='most_frequent'),[6])
     ],remainder='passthrough')

In [42]:
trf1

ColumnTransformer(remainder='passthrough',
                  transformers=[('impute_Age', SimpleImputer(), [2]),
                                ('impute_Embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

In [43]:
# One Hot Encoder Transformer 
trf2 = ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(sparse_output=False, handle_unknown='ignore'),[1,6])
    ],remainder='passthrough')

In [44]:
trf2

ColumnTransformer(remainder='passthrough',
                  transformers=[('ohe_sex_embarked',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 [1, 6])])

In [45]:
# Scaler Transformer 
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
    ])

In [46]:
trf3

ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))])

### Step 3 => Feature Selection

In [77]:
trf4 = SelectKBest(score_func=chi2, k=8)

In [80]:
trf4

SelectKBest(k=8, score_func=<function chi2 at 0x0000026A3DAAEFC0>)

### Step 4 => Train The Model

In [81]:
trf5 = DecisionTreeClassifier()

### Step 5 => Create Pipeline

In [82]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

In [83]:
pipe

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_Age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_Embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x0000026A3DAAEFC0>)),
                ('trf5', DecisionTreeClassifier())])

In [84]:
# Train Model 
pipe.fit(X_train,Y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_Age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_Embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x0000026A3DAAEFC0>)),
                ('trf5', DecisionTreeClassifier())])

In [87]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_Age', SimpleImputer(), [2]),
                                 ('impute_Embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x0000026A3DAAEFC0>),
 'trf5': DecisionTreeClassifier()}

In [88]:
Y_predict = pipe.predict(X_test)

In [89]:
Y_predict

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [90]:
accuracy_score(Y_test,Y_predict)

0.7023809523809523

### Cross Validation Using Pipeline

In [93]:
cross_val_score(pipe, X_train, Y_train, cv=5, scoring='accuracy').mean()

0.6197648123021258

### Grid Search Using Pipeline

In [95]:
params={
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [96]:
grid = GridSearchCV(pipe,params,cv=5,scoring='accuracy')

In [97]:
grid.fit(X_train,Y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_Age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_Embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x0000026A3DAAEFC0>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [98]:
grid.best_score_

0.6197648123021258

In [99]:
grid.best_params_

{'trf5__max_depth': 2}

### Exporting The Pipeline

In [102]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))